<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.30.2


    Uninstalling protobuf-6.30.2:
      Successfully uninstalled protobuf-6.30.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.60


    Uninstalling yfinance-0.2.60:
      Successfully uninstalled yfinance-0.2.60
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))
  print('-------------------')
  print('Est LTT: ',  format_amt(estimate_lt))
  print('Est LTT PnL: '+ '{} ({}%)'.format(format_amt(est_lt_pnl_amount), est_lt_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2025-05-12 09:56:31
-------------------
qualified stocks: 94
with latest results: 87
still star stocks: 47
-------------------
Initial Investment:  1.14 C
CY Investment:  1.41 C
Reserve:  3.78 L
Current:  1.25 C
-------------------
Today PnL: 3.83 L (3.15%)
Current PnL: -20.16 L (-14.29%)
CY Booked + Current PnL: -17.32 L (-12.28%)
-------------------
Total profit:  3.52 L
Total loss:  -23.68 L
-------------------
Total Booked + Current PnL: 9.66 L (8.47%)
Total Booked PnL: 29.81 L (26.14%)
Curr Year Booked PnL: 2.83 L (2.26%)
Prev Year Booked PnL: 26.98 L (23.66%)
Est FTT:  2.07 C
Est FTT PnL: 81.26 L (64.78%)
-------------------
Est LTT:  2.68 C
Est LTT PnL: 1.43 C (114.04%)
Deployed:  1.14 C
Current:  1.25 C
CAGR/XIRR %: 6.9%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,MRF,114777.60,150504.12,21.99,X-MC,92.55,140200.0,25422.0,10305.0,1.83,22.15,7.35,31.13,203.0,2.47,1.15,36.57,AR,ATH,TYRES
70,SBILIFE,1496.49,1924.99,-1.29,H-LC,87.23,146605.0,20900.0,15100.0,2.68,16.63,10.30,28.63,116.0,1.38,1.21,30.61,AR,ATH,INSURANCE
57,PIDILITIND,2504.06,3576.00,-12.26,X-LC,48.94,91494.0,16372.0,15783.0,2.34,21.79,17.25,42.81,66.0,1.04,0.75,14.74,X40,BTT,CHEMICALS
71,SBIN,760.30,863.00,-15.73,M-LC,62.77,211284.0,9804.0,17410.0,2.32,4.87,8.24,13.51,106.0,0.56,1.74,15.75,XY25,NTT,BANKS
55,NMDC,65.62,82.00,31.58,M-MC,50.00,101745.0,1346.0,23717.0,3.44,1.34,23.31,24.96,70.0,0.06,0.84,14.67,XY25,NTT,MINING


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,CERA,9059.90,10420.45,106.96,X-SC,88.30,63705.0,-26894.0,40497.0,9.23,-29.68,63.57,15.02,50.0,-0.66,0.52,25.34,X40N,ATH,CERAMICS
89,VALIANTORG,512.64,838.00,-288.40,H-SC,1.06,80067.0,-53219.0,137811.0,6.82,-39.93,172.12,63.47,106.0,-0.39,0.66,34.21,XR,NTT,CHEMICALS
83,TITAGARH,1097.23,1548.00,-27.01,H-SC,5.32,128336.0,-63679.0,142568.0,6.71,-33.16,111.09,41.08,65.0,-0.45,1.05,8.62,XY24,NTT,ENGINEERING
31,HATHWAY,19.74,31.40,878.26,L-SC,64.89,56120.0,-22840.0,69482.0,6.61,-28.93,123.81,59.07,144.0,-0.33,0.46,12.60,XR,NTT,ENTERTAINMENT
67,SAMMAANCAP,170.35,326.00,-159.50,M-SC,67.02,72714.0,-29496.0,122887.0,6.57,-28.86,169.00,91.37,103.0,-0.24,0.60,18.04,XR,NTT,FINANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
20,COFFEEDAY,59.14,80.00,-61.77,L-SC,71.81,56659.0,-56890.0,96938.0,-2.03,-50.10,171.09,35.27,151.0,-0.59,0.47,38.67,XR,NTT,HOTELS
16,BSOFT,628.85,844.81,-28.35,M-SC,81.38,67279.0,-42770.0,80567.0,-0.39,-38.86,119.75,34.34,15.0,-0.53,0.55,10.76,XR,ATH,IT
38,INDUSINDBK,1354.50,1800.00,-36.29,H-MC,86.17,46820.0,-30386.0,55781.0,0.39,-39.36,119.14,32.89,46.0,-0.54,0.38,28.94,XR,NTT,BANKS
56,PFIZER,4236.33,6318.30,-15.10,H-SC,46.81,132292.0,966.0,63580.0,0.41,0.74,48.06,49.15,89.0,0.02,1.09,9.04,X40,ATH,PHARMA
45,LALPATHLAB,2798.50,3545.00,-33.16,H-SC,44.68,128529.0,-202.0,34536.0,0.71,-0.16,26.87,26.68,37.0,-0.01,1.06,20.93,X40N,NTT,HEALTHCARE


In [12]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,TATAMOTORS,776.02,1065.00,-52.64,M-LC,95.74,190654.0,-14991.0,91571.0,1.55,-7.29,48.03,37.24,1.0,-0.16,1.57,24.10,XY24,NTT,AUTO
79,TANLA,917.30,1963.11,-57.60,H-SC,51.06,127107.0,-112308.0,385261.0,3.58,-46.91,303.10,114.01,2.0,-0.29,1.04,16.58,AR,ATH,IT
2,ABB,5319.37,8953.12,-37.83,H-LC,77.66,141062.0,8078.0,82761.0,3.71,6.07,58.67,68.31,3.0,0.10,1.16,15.02,AR,ATH,ELECTRICAL
24,DLF,683.35,1031.70,-27.44,H-LC,42.02,120276.0,-2727.0,65430.0,5.78,-2.22,54.40,50.98,4.0,-0.04,0.99,9.89,AR,ATH,REALTY
10,AWL,342.88,485.00,-68.99,H-MC,7.45,168338.0,-49391.0,139636.0,4.95,-22.68,82.95,41.45,6.0,-0.35,1.38,12.45,XY24,NTT,FMCG


In [13]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,MRF,114777.60,150504.12,21.99,X-MC,92.55,140200.0,25422.0,10305.0,1.83,22.15,7.35,31.13,203.0,2.47,1.15,36.57,AR,ATH,TYRES
57,PIDILITIND,2504.06,3576.00,-12.26,X-LC,48.94,91494.0,16372.0,15783.0,2.34,21.79,17.25,42.81,66.0,1.04,0.75,14.74,X40,BTT,CHEMICALS
4,ADANIPORTS,1103.69,1583.00,5.78,M-LC,89.36,186868.0,35662.0,30011.0,4.42,23.59,16.06,43.43,45.0,1.19,1.54,29.53,XY24,NTT,MISC


In [14]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,NMDC,65.62,82.00,31.58,M-MC,50.00,101745.0,1346.0,23717.0,3.44,1.34,23.31,24.96,70.0,0.06,0.84,14.67,XY25,NTT,MINING
58,POLYCAB,5005.92,7541.35,16.80,H-MC,90.96,165760.0,25594.0,45402.0,2.66,18.26,27.39,50.65,74.0,0.56,1.36,26.51,X40N,ATH,CABLES
4,ADANIPORTS,1103.69,1583.00,5.78,M-LC,89.36,186868.0,35662.0,30011.0,4.42,23.59,16.06,43.43,45.0,1.19,1.54,29.53,XY24,NTT,MISC
6,ANGELONE,2328.67,3033.00,-2.28,H-SC,69.15,159204.0,7840.0,37938.0,5.28,5.18,23.83,30.25,108.0,0.21,1.31,26.01,X40N,NTT,FINANCE
87,UNIONBANK,123.87,163.00,-8.12,M-LC,61.70,145024.0,4184.0,40302.0,3.78,2.97,27.79,31.59,102.0,0.10,1.19,25.98,XY24,NTT,BANKS


In [15]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
93,WIPRO,243.46,420.00,-9.26,M-LC,56.38,155868.0,4923.0,104525.0,3.88,3.26,67.06,72.51,77.0,0.05,1.28,17.36,XR,NTT,IT
18,CAMS,3643.00,5250.99,-8.32,H-SC,6.38,101461.0,-543.0,45566.0,4.14,-0.53,44.91,44.14,26.0,-0.01,0.83,17.53,XR,ATH,MISC
91,VOLTAS,1272.00,1929.20,-18.57,H-MC,21.28,138732.0,-1188.0,73486.0,2.07,-0.85,52.97,51.67,9.0,-0.02,1.14,4.79,AR,ATH,AC
12,BANKINDIA,113.49,190.00,-28.40,M-MC,25.53,125430.0,-3722.0,90786.0,2.19,-2.88,72.38,67.42,99.0,-0.04,1.03,21.99,XR,NTT,BANKS
1,AARTIIND,487.04,919.00,22.16,M-SC,97.87,77212.0,-4124.0,76262.0,2.38,-5.07,98.77,88.69,114.0,-0.05,0.63,30.20,XR,NTT,CHEMICALS


In [16]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
86,UJJIVANSFB,52.77,60.00,-41.46,M-SC,71.81,113859.0,-28620.0,48140.0,3.54,-20.09,42.28,13.70,139.0,-0.59,0.94,34.38,X40N,NTT,BANKS
52,MRF,114777.60,150504.12,21.99,X-MC,92.55,140200.0,25422.0,10305.0,1.83,22.15,7.35,31.13,203.0,2.47,1.15,36.57,AR,ATH,TYRES
87,UNIONBANK,123.87,163.00,-8.12,M-LC,61.70,145024.0,4184.0,40302.0,3.78,2.97,27.79,31.59,102.0,0.10,1.19,25.98,XY24,NTT,BANKS
70,SBILIFE,1496.49,1924.99,-1.29,H-LC,87.23,146605.0,20900.0,15100.0,2.68,16.63,10.30,28.63,116.0,1.38,1.21,30.61,AR,ATH,INSURANCE
51,MEDANTA,1022.60,1486.00,4.81,X-MC,9.57,184756.0,29321.0,41108.0,3.85,18.86,22.25,45.32,189.0,0.71,1.52,25.71,XY24,NTT,HEALTHCARE


In [17]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
90,VIPIND,488.80,718.0,-797.74,H-SC,100.00,66456.0,-28860.0,73554.0,3.13,-30.28,110.68,46.89,39.0,-0.39,0.55,31.08,X40N,NTT,MISC
80,TATAELXSI,7332.28,9000.0,-21.69,X-MC,98.94,82285.0,-20367.0,43718.0,2.43,-19.84,53.13,22.74,40.0,-0.47,0.68,23.68,X40N,NTT,IT
1,AARTIIND,487.04,919.0,22.16,M-SC,97.87,77212.0,-4124.0,76262.0,2.38,-5.07,98.77,88.69,114.0,-0.05,0.63,30.20,XR,NTT,CHEMICALS
59,QUESS,711.66,986.0,NaN,NaN,96.81,75555.0,-80299.0,140381.0,4.80,-51.52,185.80,38.55,NaN,-0.57,0.62,15.69,XY24,NTT,MISC
81,TATAMOTORS,776.02,1065.0,-52.64,M-LC,95.74,190654.0,-14991.0,91571.0,1.55,-7.29,48.03,37.24,1.0,-0.16,1.57,24.10,XY24,NTT,AUTO


In [18]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.44
1,25,41.28
2,50,69.74


In [19]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24    35.87
X40     14.57
X40N    13.80
AR      12.11
XY25    11.18
XR      10.17
X200     1.18
SR       1.11
Name: CurrAlloc%, dtype: float64

In [20]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    21.26
X-LC    18.41
M-SC    16.46
M-LC     8.47
M-MC     7.47
H-MC     6.79
X-MC     6.77
H-LC     5.96
X-SC     2.63
L-SC     2.25
Name: CurrAlloc%, dtype: float64

In [21]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
BANKS,11.07,-11.96,66.87
FMCG,11.05,-2.52,35.16
IT,9.12,-24.51,93.97
FINANCE,7.39,-23.32,78.97
MISC,6.34,-33.48,81.78
INSURANCE,6.04,-5.25,36.98
PAINTS,5.68,-19.28,54.58
ELECTRICAL,4.12,-10.98,67.53
REFINERIES,3.70,9.51,7.16


In [22]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3185831.0,28
XR,1304640.0,15
X40N,1222821.0,17
AR,1133052.0,13
X40,819991.0,12
SR,209429.0,2
XY25,176407.0,6
X200,73997.0,1


In [23]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2388023.0,21
M-SC,2000465.0,20
X-LC,741514.0,12
M-MC,683187.0,7
H-MC,400591.0,6
X-SC,356956.0,4
M-LC,350993.0,6
X-MC,330415.0,6
L-SC,324662.0,4


In [24]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      945325.0      6
M-SC       XY24      796527.0      6
H-SC       X40N      573971.0      7
M-MC       XY24      482305.0      4
H-SC       AR        447532.0      2
M-SC       XR        443645.0      5
           X40N      381540.0      5
X-LC       X40       379854.0      5
H-SC       XR        317619.0      4
M-LC       XY24      229058.0      4
M-SC       X40       199088.0      2
X-LC       AR        191363.0      3
H-MC       XY24      182501.0      2
L-SC       XR        166420.0      2
H-LC       AR        163291.0      3
X-MC       XY24      144861.0      2
X-SC       XR        125864.0      1
           SR        123548.0      1
X-MC       X40N      114364.0      2
X-SC       X40N      107544.0      2
M-LC       XR        104525.0      1
H-SC       X40       103576.0      2
X-LC       XY25       96300.0      3
M-SC       AR         93784.0      1
L-SC       XY24       91330.0      1
M-MC       XR         90786.0      1
           AR         86379.0      1
M-SC       SR         85881.0      1
X-LC       X200       73997.0      1
H-MC       AR         73486.0      1
L-SC       AR         66912.0      1
X-MC       X40        60885.0      1
H-MC       XR         55781.0      1
           X40N       45402.0      1
           X40        43421.0      1
H-LC       XY25       38980.0      1
           X40        33167.0      1
M-MC       XY25       23717.0      1
M-LC       XY25       17410.0      1
X-MC       AR         10305.0      1

In [25]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 56.0 seconds
